In [ ]:
import pandas as pd
import os
from google.cloud import translate_v3 as translate
from google.cloud import bigquery
import time

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "bubbo-dfba0-47e395cdcdc7.json"

# Autenticación para Google Cloud
translate_client = translate.TranslationServiceClient()

project_id = "bubbo-dfba0"  # ID del proyecto de Bubbo 
location = "global"  

# Conexión a BigQuery
bq_client = bigquery.Client()

def translate_text_batch(texts, project_id, location, target_language_code):
    parent = f"projects/{project_id}/locations/{location}"
    # Realizar la traducción en batch
    response = translate_client.translate_text(
        request={
            "parent": parent,
            "contents": texts,
            "mime_type": "text/plain",
            "target_language_code": target_language_code,
        }
    )
    # Extraer las traducciones
    return [translation.translated_text for translation in response.translations]

def translate_text_batch(texts, project_id, location, target_language_code):
    parent = f"projects/{project_id}/locations/{location}"
    
    # Filtra textos vacíos o no válidos
    valid_texts = [text for text in texts if isinstance(text, str) and text.strip() != '']
    
    if not valid_texts:
        return []  # Si no hay textos válidos, retorna una lista vacía
    
    # Realizar la traducción en batch
    response = translate_client.translate_text(
        request={
            "parent": parent,
            "contents": valid_texts,
            "mime_type": "text/plain",
            "target_language_code": target_language_code,
        }
    )
    
    # Extraer las traducciones
    return [translation.translated_text for translation in response.translations]


def get_data_from_bigquery(query):
    # Obtiene datos desde una tabla en BigQuery
    df = bq_client.query(query).to_dataframe()
    return df

def save_to_bigquery(df, table_id):
    # Guarda el DataFrame en BigQuery en la tabla especificada
    job_config = bigquery.LoadJobConfig(
        schema=[bigquery.SchemaField("translated_text", "STRING")],  # Ajusta el esquema según tu tabla
        write_disposition="WRITE_TRUNCATE",  # Reemplaza la tabla si ya existe
    )
    
    load_job = bq_client.load_table_from_dataframe(df, table_id, job_config=job_config)
    load_job.result()  # Espera a que el trabajo termine
    print(f"Datos cargados en {table_id}")

def main():
    # 1. Obtén datos desde BigQuery
    query = "SELECT * FROM `bubbo-dfba0.content.best_content`"  # Cambia con tu consulta y tabla
    df = get_data_from_bigquery(query)

    # 2. Traduce las columnas necesarias en batches
    columnas_a_traducir = ["genres", "title", "synopsis"]  # Ajusta las columnas según sea necesario
    translated_df = translate_df_in_batches(df, columnas_a_traducir, project_id, location)

    # 3. Guarda los resultados traducidos en una nueva tabla de BigQuery
    new_table_id = "bubbo-dfba0.content.best_content_translated"  
    save_to_bigquery(translated_df, new_table_id)

if __name__ == "__main__":
    main()
